# Deep Q-Networks

## Tabular Q-Learning
By *Tabular Q-Learning* we mean a *model-free* set of techniques which work in discrete (or discretized) environments with small amount of states and actions and keep around a table of Q values. Contrary to the Q-Value Learning from the last chapter, here we don't explicitly model the environment.

After getting an experience $(s, a, r, s')$ we perform a blending Bellman approximation update:
$$
Q(s, a) \gets (1 - \alpha) Q(s, a) + \alpha (r + \max_{a'} Q(s', a'))
$$
which can be reformulated in terms of *Temporal Difference learning (TD learning)* as
$$
Q(s, a) \gets Q(s, a) + \alpha (r + \max_{a'} Q(s', a') - Q(s, a)) = Q(s, a) + \alpha \delta(s, a, r, s')
$$
where
* $\delta(s, a, r, s') = r + \max_{a'} Q(s', a') - Q(s, a)$ is called *TD error* and
* $r + \max_{a'} Q(s', a')$ is the *TD target*

Finally, for efficiency reasons we don't actually have to construct the full Q table as we don't really care about states that we've never experience. So we'll estimate values only for those states that we've seen and iterate over a smaller set.

In [1]:
import collections
from functools import partial
from typing import Callable, NamedTuple, Tuple

import gym
import numpy as np
from tensorboardX import SummaryWriter


class Experience(NamedTuple):
    state: int
    action: int
    reward: float
    next_state: int


class Agent:
    """Q-Learning agent"""

    def __init__(
        self,
        n_actions: int,
        alpha: float,
        gamma: float,
    ) -> None:
        self.n_actions = n_actions
        self.alpha = alpha
        self.gamma = gamma
        self.values = collections.defaultdict(float)

    def policy(self, s: int) -> int:
        """Returns the best known action"""
        return np.argmax([self.values[s, a] for a in range(self.n_actions)])

    def __iadd__(self, e: Experience) -> "Agent":
        Q_max = self.values[e.next_state, self.policy(e.next_state)]
        td_target = e.reward + self.gamma * Q_max
        td_error = td_target - self.values[e.state, e.action]
        self.values[e.state, e.action] += self.alpha * td_error
        return self


def explore(env: gym.Env, state: int) -> Tuple[Experience, int]:
    """
    Samples and applies a random action in given environment from given state.
    Returns experience (s, a, r, s') and new state (resets env if necessary).
    """
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    experience = Experience(state, action, reward, next_state)
    state = env.reset() if done else next_state
    return experience, state


def evaluate(
    env: gym.Env,
    n_episodes: int,
    policy: Callable[[int], int],
) -> float:
    """
    Runs n episodes in given environment using given policy and
    computes the mean non-disounted reward.
    """

    total_reward = 0.0

    for _ in range(n_episodes):

        state = env.reset()
        episode_done = False

        while not episode_done:
            action = policy(state)
            next_state, reward, episode_done, _ = env.step(action)
            state = next_state
            total_reward += reward

    return total_reward / n_episodes


def train(
    env: gym.Env,
    eval_episodes: int = 20,
    alpha: float = 0.2,
    gamma: float = 0.9,
    solution_bound: float = 0.8,
    max_iters: int = 5_000,
) -> None:
    with SummaryWriter(comment="-q-learning") as writer:

        # Bind experience sampling and policy evaluation to environments
        #  - Note: We use a copy of the env. for testing.
        explore_env = partial(explore, env)
        eval_policy = partial(evaluate, gym.make(env.spec.id), eval_episodes)

        # Initialize the Q-Learning agent
        agent = Agent(env.action_space.n, alpha, gamma)

        i = 0
        reward = 0.0
        best_reward = 0.0

        # Initialize the environment
        state = env.reset()

        while reward < solution_bound and i < max_iters:
            i += 1

            # Sample new experience from the environment
            # and pass it to the agent to learn from it.
            experience, state = explore_env(state)
            agent += experience

            # Evaluate current policy
            mean_reward = eval_policy(agent.policy)
            best_reward = max(mean_reward, best_reward)

            # Record metrics
            writer.add_scalar("reward", mean_reward, i)
            writer.add_scalar("best_reward", best_reward, i)

    print(f"Solved in {i} iterations with best reward: {best_reward:.3f}")


# Run Q-Learning in FrozenLake
train(env=gym.make("FrozenLake-v0"))

Solved in 5000 iterations with best reward: 0.750


## Deep Q-Learning
Even Q-Learning does not iterate over all possible states, the size of the Q table might become intractable. The idea of *Deep Q-Learning* is to use a deep NN to represent $Q(s, a) \approx Q(s, a; \mathbf{w})$ ($\mathbf{w}$ are the NN parameters). We treat the problem as a supervised regression task and train the network with a SGD on a dataset of collected experiences.

### Problems in Deep Q-Learning
Although a NN model is a compact representation of the Q table, the learning dynamics we described above has some issues:

#### Exploration-Exploitation Dilema
In all the previous examples we've used random actions to sample experiences from the environment (exploration). However, if we're confident enough in the policy we have, it'd be much better to exploit it and use $a = argmax_a Q(s, a)$ in a state $s$.

A straightforward trick that is used to balance exploration and exploitation a bit is called $\epsilon$*-greedy* policy which randomly switches between these two extremes with $\epsilon$ probability. We typically define a schedule and decrease $\epsilon$ as the training proceeds.

#### SGD Optimization
SGD is heavily based on the assumption of the training instances being *i.i.d* which is definitely not the case in Q-Learning dynamics.

First, independence is broken due to the fact further states in an episode do depend on previous actions and states.

Furthermore, the training data (experiences) are not identically distributed. Regardless of which policy we use to sample the experiences (during training) it won't produce the same distibution of experiences as if we had used the optimal policy. In short, our targets do not follow the same distribution as what we're trying to learn.

To mitigate the i.i.d problem we introduce a *replay buffer* - a large collection of experinces from which we sample training mini-batches.

#### Experience Correlations
Next issue is that the future Q values $Q(s', a')$ are highly correlated with current $Q(s, a)$ that we try to update. The model starts by modifying $Q(s', a')$ which might cause $Q(s, a)$ to get worse but then by trying fixing $Q(s, a)$ we break $Q(s', a')$. Basically, the training can be quite unstable.

A technique that is frequently used to address this is described as *fixing (freezing) the targets*. We introduce second NN $\hat{Q}$ which is a copy of our $Q$ NN and use DT targets $r + \gamma \max_{a'} \hat{Q}(s', a')$. NN parameters are then regularly synchronized with quite a large period (1k or even 10k iterations).

### DQN Training
Putting all the pieces together, here is an overview of the training algorithm for a *Deep Q-Network (DQN)*.
1. Initialize parameters of $Q$ and $\hat{Q}$, start with $\epsilon \gets 1$
1. Select a random action $a$ with $\epsilon$ probability, otherwise pick $a = argmax_a Q(s, a)$
1. Run $a$ in the environment and gain experience $e = (s, a, r, s')$
1. Store $e$ in the replay buffer
1. Sample random mini-batch $B$ from the replay buffer
1. For each $(s, a, r, s') \in B$ alculate targets $y$: $y = r$ if episode ends, otherwise $y = r + \gamma \max_{a'} \hat{Q}(s', a')$
1. Compute the loss $\mathcal{L} = (Q(s, a) - y)^2$
1. Using $\mathcal{L}$ run a SGD step to update $Q(s, a)$
1. Every $N$ steps copy $\mathbf{w}_Q \to \mathbf{w}_{\hat{Q}}$
1. Repeat from 2. until convergence or other termination condition